In [1]:
import pandas as pd
import pyreadr
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import warnings

In [2]:
database1 = pd.read_csv("/Users/rishi/Documents/ORNL Internships/25/Data/GSTI_CSVs/Database_p1.csv", encoding='latin1')
database2 = pd.read_csv("/Users/rishi/Documents/ORNL Internships/25/Data/GSTI_CSVs/Database_p2.csv", encoding='latin1')
database3 = pd.read_csv("/Users/rishi/Documents/ORNL Internships/25/Data/GSTI_CSVs/Database_p3.csv", encoding='latin1')

database = pd.concat([database1,database2,database3],ignore_index=True)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1,29,30,39,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1,29,30,33,35,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (29,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
path = '/Users/rishi/Downloads/1_QC_ACi_data.Rdata'

In [23]:

# Suppress warnings from optimization, which can be common
warnings.filterwarnings("ignore", message="Desired error not necessarily achieved due to precision loss.")
warnings.filterwarnings("ignore", message="Values in x were outside bounds during a minimize step, clipping to bounds.")

#==============================================================================
# Helper Functions (Temperature Dependencies and Smoothing)
#==============================================================================

def f_arrhenius(PRef, Ha, Tleaf, TRef=298.15, R=8.314):
    """Arrhenius function for temperature dependence."""
    return PRef * np.exp(Ha * (Tleaf - TRef) / (TRef * R * Tleaf))

def f_modified_arrhenius(PRef, Ha, Hd, s, Tleaf, TRef=298.15, R=8.314):
    """
    Modified Arrhenius function with a deactivation term.
    Includes a check for a large Hd value to disable the deactivation term,
    preventing overflow errors and NaN generation.
    """
    term1 = f_arrhenius(PRef, Ha, Tleaf, TRef, R)
    
    # If Hd is very large (>200,000), it's being used as a sentinel value
    # to indicate that there is no high-temperature deactivation.
    # In this case, we bypass the deactivation calculation entirely.
    if Hd > 200000:
        return term1

    # Original calculation for when deactivation is active
    term2 = 1 + np.exp((s * TRef - Hd) / (R * TRef))
    term3 = 1 + np.exp((s * Tleaf - Hd) / (R * Tleaf))
    
    return term1 * term2 / term3

def f_smooth(A1, A2, theta):
    """Smoothing function to handle transitions between limiting states."""
    # The R code contains two solutions (roots). The first one is for the minimum.
    return ((A1 + A2) - np.sqrt((A1 + A2)**2 - 4 * theta * A1 * A2)) / (2 * theta)

#==============================================================================
# Core Photosynthesis Model (f.ACi)
#==============================================================================

def f_make_param(VcmaxRef=60, JmaxRef=102, RdayRef=2, TPURef=5):
    """Creates a dictionary of default parameters for the FvCB model."""
    return {
        'R': 8.314, 'O2': 210000, 'TRef': 298.15, 'Patm': 101000,
        'JmaxRef': JmaxRef, 'JmaxHa': 45830, 'JmaxHd': 152041, 'JmaxS': 496,
        'VcmaxRef': VcmaxRef, 'VcmaxHa': 65710, 'VcmaxHd': 149252, 'VcmaxS': 485,
        'TPURef': TPURef, 'TPUHa': 37000, 'TPUHd': 999999, 'TPUS': 9999,
        'thetacj': 0.98, 'thetaip': 0.98,
        'RdayRef': RdayRef, 'RdayHa': 46390, 'RdayHd': 999999, 'RdayS': 9999,
        'KcRef': 27.23, 'KcHa': 80990, 'KoRef': 16582, 'KoHa': 15110,
        'GstarRef': 4.33, 'GstarHa': 37830,
        'abso': 0.8, 'aQY': 0.3, 'Theta': 0.83
    }


#==============================================================================
# Fitting and Plotting Functions
#==============================================================================
def f_sum_sq(x, names, data, fixed_param):
    """Objective function for least squares: calculates sum of squared residuals."""
    fit_param = dict(zip(names, x))
    param = {**fixed_param, **fit_param}
    A_pred = f_aci(Ci=data['Ci'].values, PFD=data['Qin'].values, Tleaf=data['Tleaf'].values, param=param)['A']
    return np.sum((data['A'].values - A_pred)**2)

def f_minus_logL(x, names, data, fixed_param):
    """Objective function for MLE: calculates negative log-likelihood."""
    # The first parameter is always sigma (std dev of residuals)
    sigma = x[0]
    p_fit = x[1:]
    
    fit_param = dict(zip(names, p_fit))
    param = {**fixed_param, **fit_param}
    
    A_pred = f_aci(Ci=data['Ci'].values, PFD=data['Qin'].values, Tleaf=data['Tleaf'].values, param=param)['A']
    
    # Log-likelihood of normal distribution
    logL = -np.sum(np.log(2 * np.pi * sigma**2) / 2 + (data['A'].values - A_pred)**2 / (2 * sigma**2))
    
    # Return the negative log-likelihood for minimization
    return -logL

def f_fitting(measures, start_params, param):
    """
    Performs the two-stage fitting procedure for a single curve.
    1. Minimize sum of squares to get good initial guesses.
    2. Minimize negative log-likelihood (MLE) for final parameters and stats.
    """
    fixed = {k: v for k, v in param.items() if k not in start_params}
    param_names = list(start_params.keys())
    initial_guess = list(start_params.values())

    # Stage 1: Least Squares Optimization
    # The R code uses a grid search; for simplicity, we start from the provided guess.
    # A more robust version could implement a similar grid search here.
    lsq_result = minimize(f_sum_sq, x0=initial_guess, args=(param_names, measures, fixed),
                          method='L-BFGS-B', bounds=[(0, None)]*len(initial_guess))
    
    if not lsq_result.success:
        return None

    # Use LSQ results as starting point for MLE
    mle_start_params = lsq_result.x
    
    # Stage 2: Maximum Likelihood Estimation
    n_obs = len(measures)
    initial_sigma = np.sqrt(lsq_result.fun / n_obs)
    mle_initial_guess = [initial_sigma] + list(mle_start_params)
    mle_param_names = list(start_params.keys())
    
    # Set bounds (sigma > 0, others > 0)
    mle_bounds = [(1e-6, None)] + [(0, None)]*len(mle_param_names)

    mle_result = minimize(f_minus_logL, x0=mle_initial_guess, args=(mle_param_names, measures, fixed),
                          method='L-BFGS-B', bounds=mle_bounds)
    
    if not mle_result.success:
        return None
        
    # Extract results
    k = len(mle_result.x) # Number of estimated parameters (including sigma)
    logL = -mle_result.fun
    aic = 2 * k - 2 * logL
    
    # Extract coefficients
    final_coeffs = dict(zip(['sigma'] + mle_param_names, mle_result.x))
    
    # Approximate standard errors from the inverse Hessian matrix
    # This is a key feature of MLE.
    try:
        inv_hessian = np.linalg.inv(mle_result.hess_inv.todense())
        std_errors = np.sqrt(np.diag(inv_hessian))
        final_std_errors = dict(zip(['StdError_sigma'] + [f'StdError_{n}' for n in mle_param_names], std_errors))
    except (np.linalg.LinAlgError, AttributeError):
        # If Hessian is not invertible, cannot calculate std errors
        final_std_errors = dict(zip([f'StdError_{n}' for n in ['sigma'] + mle_param_names], [np.nan] * k))

    return {
        'coeffs': final_coeffs,
        'std_errors': final_std_errors,
        'aic': aic,
        'logL': logL
    }
    
def f_plot(measures, fit_params, name):
    """Plots the A-Ci data and the fitted model."""
    param = f_make_param()
    param.update(fit_params)
    
    ci_range = np.linspace(min(measures['Ci']), max(measures['Ci']), 200)
    sim = f_aci(Ci=ci_range, PFD=measures['Qin'].mean(), Tleaf=measures['Tleaf'].mean(), param=param)
    
    plt.figure(figsize=(8, 6))
    plt.scatter(measures['Ci'], measures['A'], facecolors='none', edgecolors='black', label='Observed')
    plt.plot(ci_range, sim['A'], color='darkgrey', lw=2, label='A (Best Fit)')
    plt.plot(ci_range, sim['Ac'], color='darkblue', linestyle='--', label='$A_c$')
    plt.plot(ci_range, sim['Aj'], color='darkred', linestyle='--', label='$A_j$')
    plt.plot(ci_range, sim['Ap'], color='darkgreen', linestyle='--', label='$A_p$')
    
    plt.title(f'A-Ci Curve Fit: {name}')
    plt.xlabel('$C_i$ (ppm)')
    plt.ylabel('A (μmol m$^{-2}$ s$^{-1}$)')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.show()


#==============================================================================
# Main Wrapper Function (f.fit_ACi)
#==============================================================================

def f_aci(Ci, PFD, Tleaf, param):
    """
    Calculates net photosynthesis (A) based on the FvCB model.
    Tleaf should be in Kelvin.
    """
    # Temperature dependence of kinetic parameters
    Kc = f_arrhenius(param['KcRef'], param['KcHa'], Tleaf)
    Ko = f_arrhenius(param['KoRef'], param['KoHa'], Tleaf)
    Gstar = f_arrhenius(param['GstarRef'], param['GstarHa'], Tleaf)

    # Temperature dependence of Vcmax, Jmax, TPU, and Rday
    Vcmax = f_modified_arrhenius(param['VcmaxRef'], param['VcmaxHa'], param['VcmaxHd'], param['VcmaxS'], Tleaf)
    Jmax = f_modified_arrhenius(param['JmaxRef'], param['JmaxHa'], param['JmaxHd'], param['JmaxS'], Tleaf)
    TPU = f_modified_arrhenius(param['TPURef'], param['TPUHa'], param['TPUHd'], param['TPUS'], Tleaf)
    Rday = f_modified_arrhenius(param['RdayRef'], param['RdayHa'], param['RdayHd'], param['RdayS'], Tleaf)
    
    # Electron transport rate (J)
    I2 = PFD * param['abso'] * param['aQY']
    J = (I2 + Jmax - np.sqrt((I2 + Jmax)**2 - 4 * param['Theta'] * I2 * Jmax)) / (2 * param['Theta'])

    # Photosynthesis limiting rates
    Wc = Vcmax * (Ci - Gstar) / (Ci + Kc * (1 + param['O2'] / Ko))
    Wj = (J / 4) * (Ci - Gstar) / (Ci + 2 * Gstar)
    Wp = 3 * TPU

    # Smooth transitions and final assimilation rate (A)
    Ai = f_smooth(Wc, Wj, param['thetacj'])
    A = f_smooth(Ai, Wp, param['thetaip']) - Rday
    
    # Ensure physically possible values
    # FIX: This assignment must handle cases where Tleaf (and thus Rday)
    # is a scalar or an array.
    mask = Ci <= Gstar
    if np.isscalar(Rday):
        A[mask] = -Rday
    else:
        # If Rday is an array, we must use the same boolean mask.
        A[mask] = -Rday[mask]

    return {
        'A': A,
        'Ac': Wc - Rday,
        'Aj': Wj - Rday,
        'Ap': Wp - Rday
    }

def f_fit_aci(measures, VcmaxRef=60, JmaxRef=120, RdayRef=2, TPURef=5, plot_fits=True):
    """
    Main function to fit A-Ci curves for all samples in a dataframe.
    """
    if measures[['A', 'Ci', 'Tleaf', 'Qin', 'SampleID_num']].isnull().values.any():
        raise ValueError("Input columns have NA values")

    # Convert Tleaf to Kelvin for calculations
    measures['Tleaf'] = measures['Tleaf'] + 273.15
    
    all_results = []
    
    for sample_id, curve_data in measures.groupby('SampleID_num'):
        #print(f"\n--- Fitting SampleID_num: {sample_id} ---")
        param_defaults = f_make_param()
        model_fits = {}

        models_to_fit = {
            'Ac_Aj_Ap': {'VcmaxRef': VcmaxRef, 'JmaxRef': JmaxRef, 'TPURef': TPURef, 'RdayRef': RdayRef},
            'Ac_Aj':    {'VcmaxRef': VcmaxRef, 'JmaxRef': JmaxRef, 'RdayRef': RdayRef},
            'Ac_Ap':    {'VcmaxRef': VcmaxRef, 'TPURef': TPURef, 'RdayRef': RdayRef},
            'Ac':       {'VcmaxRef': VcmaxRef, 'RdayRef': RdayRef}
        }
        
        for model_name, start_params in models_to_fit.items():
            #print(f"  Testing model: {model_name}")
            fit_result = f_fitting(curve_data.copy(), start_params, param_defaults)
            if fit_result:
                model_fits[model_name] = fit_result
                #print(f"    -> AIC: {fit_result['aic']:.2f}")
            else:
                 #print(f"    -> {model_name} fit failed.")
                continue

        if not model_fits:
            print(f"  All fits failed for sample {sample_id}.")
            continue
            
        best_model_name = min(model_fits, key=lambda m: model_fits[m]['aic'])
        best_fit = model_fits[best_model_name]
        
        #print(f"  Best model for sample {sample_id}: {best_model_name} (AIC: {best_fit['aic']:.2f})")

        final_row = {
            'SampleID_num': sample_id, 'Model': best_model_name, 'AIC': best_fit['aic'],
            'Tleaf_C': curve_data['Tleaf'].mean() - 273.15,
            'Qin_mean': curve_data['Qin'].mean(),
            **best_fit['coeffs'], **best_fit['std_errors']
        }
        all_results.append(final_row)
        
        #if plot_fits:
            #f_plot(curve_data, best_fit['coeffs'], f"Sample {sample_id} (Model: {best_model_name})")

    bilan = pd.DataFrame(all_results)
    column_map = {'VcmaxRef': 'Vcmax25', 'JmaxRef': 'Jmax25', 'TPURef': 'TPU25', 'RdayRef': 'Rday25'}
    bilan = bilan.rename(columns=column_map)
    
    # Rename std error columns and ensure all columns exist
    for old_name, new_name in column_map.items():
        bilan = bilan.rename(columns={f'StdError_{old_name}': f'StdError_{new_name}'})
        if new_name not in bilan.columns: bilan[new_name] = np.nan
        if f'StdError_{new_name}' not in bilan.columns: bilan[f'StdError_{new_name}'] = np.nan

    final_cols = ["SampleID_num", "Model", "Vcmax25", "Jmax25", "TPU25", "Rday25", 
                  "StdError_Vcmax25", "StdError_Jmax25", "StdError_TPU25", "StdError_Rday25", 
                  "Tleaf_C", "Qin_mean", "AIC", "sigma"]
    
    return bilan[[c for c in final_cols if c in bilan.columns]]


# 1) Load & sort your data as before



   SampleID_num     Model    Vcmax25      Jmax25       TPU25    Rday25  \
0           3.0     Ac_Ap  43.948022         NaN    5.432570  3.316471   
1           4.0  Ac_Aj_Ap  69.377092  149.142855  220.445721  6.026857   
2           5.0        Ac  17.965558         NaN         NaN  1.752470   
3           6.0     Ac_Ap  28.474690         NaN    4.865074  2.705365   
4           7.0  Ac_Aj_Ap  25.865687   83.094481  431.099861  2.690305   

   StdError_Vcmax25  StdError_Jmax25  StdError_TPU25  StdError_Rday25  \
0          1.000000              NaN        1.000000         1.000000   
1          1.000000          1.00000        1.000000         1.000000   
2          1.000000              NaN             NaN         1.000000   
3          1.000000              NaN        1.000000         1.000000   
4          5.648758          1.57842        1.161451        25.515937   

     Tleaf_C     Qin_mean        AIC     sigma  
0  30.495000   999.250000  20.598249  0.409012  
1  29.979167  1999

In [29]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import warnings

# Suppress warnings from optimization, which can be common
warnings.filterwarnings("ignore", message="Desired error not necessarily achieved due to precision loss.")
warnings.filterwarnings("ignore", message="Values in x were outside bounds during a minimize step, clipping to bounds.")

#==============================================================================
# Helper Functions (Temperature Dependencies and Smoothing)
#==============================================================================
def f_arrhenius(PRef, Ha, Tleaf, TRef=298.16, R=8.314): # TRef changed to 298.16 to match R
    """Arrhenius function for temperature dependence."""
    # This form is mathematically equivalent to R's PRef * exp(Ha/(R*TRef)-Ha/(R*Tleaf))
    return PRef * np.exp(Ha * (Tleaf - TRef) / (TRef * R * Tleaf))

def f_modified_arrhenius(PRef, Ha, Hd, s, Tleaf, TRef=298.16, R=8.314): # TRef changed to 298.16 to match R
    """
    Modified Arrhenius function with a deactivation term, matching the R implementation's structure.
    Includes a check for a large Hd value to disable the deactivation term,
    preventing overflow errors and NaN generation.
    """
    # This directly implements the R formula's structure for the Arrhenius part
    arrhenius_term = PRef * np.exp(Ha * (1/(R * TRef) - 1/(R * Tleaf)))

    # If Hd is very large (>200,000), it's being used as a sentinel value
    # to indicate that there is no high-temperature deactivation.
    if Hd > 200000: # Retained the sentinel value check
        return arrhenius_term # No deactivation term

    # Original calculation for when deactivation is active
    deactivation_term_numerator = (1 + np.exp((s * TRef - Hd) / (R * TRef)))
    deactivation_term_denominator = (1 + np.exp((s * Tleaf - Hd) / (R * Tleaf)))

    return arrhenius_term * deactivation_term_numerator / deactivation_term_denominator

def f_smooth(A1, A2, theta, root=1):
    """Smoothing function to handle transitions between limiting states.
    Includes root argument to match R's implementation.
    """
    discriminant = (A1 + A2)**2 - 4 * theta * A1 * A2
    # Ensure the term under the sqrt is non-negative
    discriminant = np.maximum(discriminant, 0)

    if root == 1:
        sol = ((A1 + A2) - np.sqrt(discriminant)) / (2 * theta)
    else: # root == 2
        sol = ((A1 + A2) + np.sqrt(discriminant)) / (2 * theta)
    return sol

#==============================================================================
# Core Photosynthesis Model (f.ACi)
#==============================================================================
def f_make_param(VcmaxRef=None, JmaxRef=None, RdayRef=None, TPURef=None):
    """
    Creates a dictionary of default parameters for the FvCB model,
    aligned with R's f.make.param defaults.
    """
    param = {
        'R': 8.314, 'O2': 210, 'TRef': 298.16, 'Patm': 101, # O2 and Patm are aligned with R's effective values
        'JmaxRef': 83.5, 'JmaxHa': 43540, 'JmaxHd': 152040, 'JmaxS': 495,
        'VcmaxRef': 50, 'VcmaxHa': 65330, 'VcmaxHd': 149250, 'VcmaxS': 485,
        'TPURef': (1/6) * 50, 'TPUHa': 53100, 'TPUHd': 150650, 'TPUS': 490,
        'thetacj': 0.999, 'thetaip': 0.999,
        'RdayRef': 1.43, 'RdayHa': 46390, 'RdayHd': 150650, 'RdayS': 490,
        'KcRef': 404.9, 'KcHa': 79430, 'KoRef': 278.4, 'KoHa': 36380,
        'GstarRef': 42.75, 'GstarHa': 37830,
        'abso': 0.85, 'aQY': 0.425, 'Theta': 0.7
    }

    # Override defaults with any provided arguments
    if VcmaxRef is not None:
        param['VcmaxRef'] = VcmaxRef
    if JmaxRef is not None:
        param['JmaxRef'] = JmaxRef
    if RdayRef is not None:
        param['RdayRef'] = RdayRef
    if TPURef is not None:
        param['TPURef'] = TPURef
    
    return param

def f_aci(Ci, PFD, Tleaf, param):
    """
    Calculates net photosynthesis (A) based on the FvCB model.
    Tleaf should be in Kelvin.
    """
    # Temperature dependence of kinetic parameters
    Kc = f_arrhenius(param['KcRef'], param['KcHa'], Tleaf, TRef=param['TRef'], R=param['R'])
    Ko = f_arrhenius(param['KoRef'], param['KoHa'], Tleaf, TRef=param['TRef'], R=param['R'])
    Gstar = f_arrhenius(param['GstarRef'], param['GstarHa'], Tleaf, TRef=param['TRef'], R=param['R'])
    
    # Rday, Vcmax, Jmax, TPU use the modified Arrhenius
    Rday = f_modified_arrhenius(param['RdayRef'], param['RdayHa'], param['RdayHd'], param['RdayS'], Tleaf, TRef=param['TRef'], R=param['R'])
    Vcmax = f_modified_arrhenius(param['VcmaxRef'], param['VcmaxHa'], param['VcmaxHd'], param['VcmaxS'], Tleaf, TRef=param['TRef'], R=param['R'])
    Jmax = f_modified_arrhenius(param['JmaxRef'], param['JmaxHa'], param['JmaxHd'], param['JmaxS'], Tleaf, TRef=param['TRef'], R=param['R'])
    TPU = f_modified_arrhenius(param['TPURef'], param['TPUHa'], param['TPUHd'], param['TPUS'], Tleaf, TRef=param['TRef'], R=param['R'])

    # Electron transport rate (J)
    I2 = PFD * param['abso'] * param['aQY']
    # Ensure discriminant for J is non-negative
    discriminant_J = (I2 + Jmax)**2 - 4 * param['Theta'] * I2 * Jmax
    J = (I2 + Jmax - np.sqrt(np.maximum(discriminant_J, 0))) / (2 * param['Theta'])

    # Photosynthesis limiting rates
    Wc = Vcmax * (Ci - Gstar) / (Ci + Kc * (1 + param['O2'] / Ko))
    Wj = (J / 4) * (Ci - Gstar) / (Ci + 2 * Gstar)
    Wp = 3 * TPU

    # Smooth transitions and final assimilation rate (A)
    # Apply smoothing with appropriate root based on Ci and Gstar
    Ai = np.zeros_like(Ci, dtype=float)
    
    # For Ci > Gstar, use root=1 (standard solution)
    mask_ci_gt_gstar = Ci > Gstar
    Ai[mask_ci_gt_gstar] = f_smooth(Wc[mask_ci_gt_gstar], Wj[mask_ci_gt_gstar], param['thetacj'], root=1)
    
    # For Ci <= Gstar, use root=2 (as per R's f.ACi implementation for this condition)
    mask_ci_le_gstar = Ci <= Gstar
    Ai[mask_ci_le_gstar] = f_smooth(Wc[mask_ci_le_gstar], Wj[mask_ci_le_gstar], param['thetacj'], root=2)
    
    A = f_smooth(Ai, Wp, param['thetaip']) - Rday

    # Ensure physically possible values for A when Ci <= Gstar
    mask_negative_Ci_effect = Ci <= Gstar
    if np.isscalar(Rday):
        A[mask_negative_Ci_effect] = -Rday
    else:
        A[mask_negative_Ci_effect] = -Rday[mask_negative_Ci_effect]

    return {
        'A': A,
        'Ac': Wc - Rday,
        'Aj': Wj - Rday,
        'Ap': Wp - Rday
    }

#==============================================================================
# Fitting and Plotting Functions
#==============================================================================
def f_sum_sq(x, names, data, fixed_param):
    """Objective function for least squares: calculates sum of squared residuals."""
    fit_param = dict(zip(names, x))
    param = {**fixed_param, **fit_param}
    A_pred = f_aci(Ci=data['Ci'].values, PFD=data['Qin'].values, Tleaf=data['Tleaf'].values, param=param)['A']
    return np.sum((data['A'].values - A_pred)**2)

def f_minus_logL(x, names, data, fixed_param):
    """Objective function for MLE: calculates negative log-likelihood."""
    # The first parameter is always sigma (std dev of residuals)
    sigma = x[0]
    p_fit = x[1:]
    fit_param = dict(zip(names, p_fit))
    param = {**fixed_param, **fit_param}
    A_pred = f_aci(Ci=data['Ci'].values, PFD=data['Qin'].values, Tleaf=data['Tleaf'].values, param=param)['A']
    
    # Log-likelihood of normal distribution
    # Ensure sigma is positive for log and division
    sigma = np.maximum(sigma, 1e-9) # Small positive lower bound
    logL = -np.sum(np.log(np.sqrt(2 * np.pi * sigma**2)) + (data['A'].values - A_pred)**2 / (2 * sigma**2))
    
    # Return the negative log-likelihood for minimization
    return -logL

def f_fitting(measures, start_params, param):
    """
    Performs the two-stage fitting procedure for a single curve.
    1. Minimize sum of squares to get good initial guesses.
    2. Minimize negative log-likelihood (MLE) for final parameters and stats.
    """
    fixed = {k: v for k, v in param.items() if k not in start_params}
    param_names = list(start_params.keys())
    initial_guess = list(start_params.values())

    # Stage 1: Least Squares Optimization
    # The R code uses a grid search; for simplicity, we start from the provided guess.
    # A more robust version could implement a similar grid search here.
    lsq_result = minimize(f_sum_sq, x0=initial_guess, args=(param_names, measures, fixed),
                          method='L-BFGS-B', bounds=[(0, None)]*len(initial_guess))
    
    if not lsq_result.success:
        print(f"Least Squares Optimization failed: {lsq_result.message}")
        return None

    # Use LSQ results as starting point for MLE
    mle_start_params = lsq_result.x

    # Stage 2: Maximum Likelihood Estimation
    n_obs = len(measures)
    initial_sigma = np.sqrt(lsq_result.fun / n_obs)
    mle_initial_guess = [initial_sigma] + list(mle_start_params)
    mle_param_names = list(start_params.keys())

    # Set bounds (sigma > 0, others > 0)
    mle_bounds = [(1e-6, None)] + [(0, None)]*len(mle_param_names)
    
    mle_result = minimize(f_minus_logL, x0=mle_initial_guess, args=(mle_param_names, measures, fixed),
                          method='L-BFGS-B', bounds=mle_bounds)
    
    if not mle_result.success:
        print(f"MLE Optimization failed: {mle_result.message}")
        return None

    # Extract results
    k = len(mle_result.x) # Number of estimated parameters (including sigma)
    logL = -mle_result.fun
    aic = 2 * k - 2 * logL

    # Extract coefficients
    final_coeffs = dict(zip(['sigma'] + mle_param_names, mle_result.x))

    # Approximate standard errors from the inverse Hessian matrix
    # Correction: mle_result.hess_inv is already the approximate covariance matrix.
    try:
        covariance_matrix = mle_result.hess_inv.todense()
        std_errors = np.sqrt(np.diag(covariance_matrix))
        final_std_errors = dict(zip(['StdError_sigma'] + [f'StdError_{n}' for n in mle_param_names], std_errors))
    except (np.linalg.LinAlgError, AttributeError):
        # If Hessian is not invertible or hess_inv is not available/dense, cannot calculate std errors
        final_std_errors = dict(zip([f'StdError_{n}' for n in ['sigma'] + mle_param_names], [np.nan] * k))

    return {
        'coeffs': final_coeffs,
        'std_errors': final_std_errors,
        'aic': aic,
        'logL': logL
    }

def f_plot(measures, fit_params, name):
    """Plots the A-Ci data and the fitted model."""
    param = f_make_param()
    param.update(fit_params)
    ci_range = np.linspace(min(measures['Ci']), max(measures['Ci']), 200)
    sim = f_aci(Ci=ci_range, PFD=measures['Qin'].mean(), Tleaf=measures['Tleaf'].mean(), param=param)
    plt.figure(figsize=(8, 6))
    plt.scatter(measures['Ci'], measures['A'], facecolors='none', edgecolors='black', label='Observed')
    plt.plot(ci_range, sim['A'], color='darkgrey', lw=2, label='A (Best Fit)')
    plt.plot(ci_range, sim['Ac'], color='darkblue', linestyle='--', label='$A_c$')
    plt.plot(ci_range, sim['Aj'], color='darkred', linestyle='--', label='$A_j$')
    plt.plot(ci_range, sim['Ap'], color='darkgreen', linestyle='--', label='$A_p$')
    plt.title(f'A-Ci Curve Fit: {name}')
    plt.xlabel('$C_i$ (ppm)')
    plt.ylabel('A (μmol m$^{-2}$ s$^{-1}$)')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.show()

#==============================================================================
# Main Wrapper Function (f.fit_ACi)
#==============================================================================
def f_fit_aci(measures, VcmaxRef=50, JmaxRef=83.5, RdayRef=1.43, TPURef=(1/6)*50, plot_fits=True):
    """
    Main function to fit A-Ci curves for all samples in a dataframe.
    Default VcmaxRef, JmaxRef, RdayRef, TPURef are set to match R's f.make.param defaults
    for initial 'start_params'.
    """
    if measures[['A', 'Ci', 'Tleaf', 'Qin', 'SampleID_num']].isnull().values.any():
        raise ValueError("Input columns have NA values")

    # Convert Tleaf to Kelvin for calculations
    measures['Tleaf'] = measures['Tleaf'] + 273.15

    all_results = []
    for sample_id, curve_data in measures.groupby('SampleID_num'):
        #print(f"\n--- Fitting SampleID_num: {sample_id} ---")
        param_defaults = f_make_param() # Get the R-aligned default parameters
        model_fits = {}

        # Default start parameters for each model type, matching f.make.param defaults
        models_to_fit = {
            'Ac_Aj_Ap': {'VcmaxRef': VcmaxRef, 'JmaxRef': JmaxRef, 'TPURef': TPURef, 'RdayRef': RdayRef},
            'Ac_Aj': {'VcmaxRef': VcmaxRef, 'JmaxRef': JmaxRef, 'RdayRef': RdayRef},
            'Ac_Ap': {'VcmaxRef': VcmaxRef, 'TPURef': TPURef, 'RdayRef': RdayRef},
            'Ac': {'VcmaxRef': VcmaxRef, 'RdayRef': RdayRef}
        }

        for model_name, start_params in models_to_fit.items():
            #print(f" Testing model: {model_name}")
            fit_result = f_fitting(curve_data.copy(), start_params, param_defaults)
            if fit_result:
                model_fits[model_name] = fit_result
                #print(f" -> AIC: {fit_result['aic']:.2f}")
            else:
                #print(f" -> {model_name} fit failed.")
                continue

        if not model_fits:
            print(f" All fits failed for sample {sample_id}.")
            continue

        best_model_name = min(model_fits, key=lambda m: model_fits[m]['aic'])
        best_fit = model_fits[best_model_name]
        #print(f" Best model for sample {sample_id}: {best_model_name} (AIC: {best_fit['aic']:.2f})")

        final_row = {
            'SampleID_num': sample_id, 'Model': best_model_name, 'AIC': best_fit['aic'],
            'Tleaf_C': curve_data['Tleaf'].mean() - 273.15,
            'Qin_mean': curve_data['Qin'].mean(),
            **best_fit['coeffs'], **best_fit['std_errors']
        }
        all_results.append(final_row)

        #if plot_fits: # Uncomment if you want to plot each fit
            #f_plot(curve_data, best_fit['coeffs'], f"Sample {sample_id} (Model: {best_model_name})")

    bilan = pd.DataFrame(all_results)
    
    # Rename columns to match R output for consistency
    column_map = {'VcmaxRef': 'Vcmax25', 'JmaxRef': 'Jmax25', 'TPURef': 'TPU25', 'RdayRef': 'Rday25'}
    bilan = bilan.rename(columns=column_map)

    # Rename std error columns and ensure all expected columns exist
    for old_name, new_name in column_map.items():
        bilan = bilan.rename(columns={f'StdError_{old_name}': f'StdError_{new_name}'})
        # Ensure the column exists even if it's all NaN from a failed std error calc
        if new_name not in bilan.columns:
            bilan[new_name] = np.nan
        if f'StdError_{new_name}' not in bilan.columns:
            bilan[f'StdError_{new_name}'] = np.nan

    final_cols = ["SampleID_num", "Model", "Vcmax25", "Jmax25", "TPU25", "Rday25", 
                  "StdError_Vcmax25", "StdError_Jmax25", "StdError_TPU25", "StdError_Rday25", 
                  "Tleaf_C", "Qin_mean", "AIC", "sigma"]
    
    # Select and reorder columns, dropping any that weren't generated (e.g., if a parameter wasn't fitted)
    return bilan[[c for c in final_cols if c in bilan.columns]]

# Example usage (assuming 'path' to curated_data.RData is defined)
# import pyreadr
# path = 'path/to/your/curated_data.RData' # Define your path here
# result = pyreadr.read_r(path)
# curated_data = result['curated_data']

# Create dummy data for testing if no .RData file is available
# This dummy data will not produce meaningful fits but allows the code to run

import pyreadr
import pandas as pd

result = pyreadr.read_r(path)
curated_data = result['curated_data']
curated_data = curated_data.sort_values(
    by=['SampleID_num', 'Ci']
).reset_index(drop=True)

# 2) Fit all curves using the f_fit_aci defaults:
bilan = f_fit_aci(curated_data)

# 3) Inspect the results
print(bilan.head())

Least Squares Optimization failed: ABNORMAL_TERMINATION_IN_LNSRCH
MLE Optimization failed: ABNORMAL_TERMINATION_IN_LNSRCH
   SampleID_num     Model    Vcmax25      Jmax25     TPU25    Rday25  \
0           3.0  Ac_Aj_Ap  61.233232   87.032902  4.135472  0.213945   
1           4.0  Ac_Aj_Ap  99.550413  141.862255  8.076031  1.048029   
2           5.0     Ac_Aj  22.208918   53.093089       NaN  0.183583   
3           6.0     Ac_Aj  39.081798   64.337725       NaN  0.585925   
4           7.0     Ac_Aj  32.883071   69.594801       NaN  0.700197   

   StdError_Vcmax25  StdError_Jmax25  StdError_TPU25  StdError_Rday25  \
0               1.0              1.0             1.0              1.0   
1               1.0              1.0             1.0              1.0   
2               1.0              1.0             NaN              1.0   
3               1.0              1.0             NaN              1.0   
4               1.0              1.0             NaN              1.0   

     T

In [30]:
bilan

,SampleID_num,Model,Vcmax25,Jmax25,TPU25,Rday25,StdError_Vcmax25,StdError_Jmax25,StdError_TPU25,StdError_Rday25,Tleaf_C,Qin_mean,AIC,sigma
0,3.0,Ac_Aj_Ap,61.233232,87.032902,4.135472,0.213945,1.0,1.0,1.0,1.0,30.495000,999.250000,7.890346,0.221609
1,4.0,Ac_Aj_Ap,99.550413,141.862255,8.076031,1.048029,1.0,1.0,1.0,1.0,29.979167,1999.916667,19.159461,0.354412
2,5.0,Ac_Aj,22.208918,53.093089,NaN,0.183583,1.0,1.0,NaN,1.0,29.891818,999.272727,13.487707,0.310523
3,6.0,Ac_Aj,39.081798,64.337725,NaN,0.585925,1.0,1.0,NaN,1.0,29.907857,1000.071429,2.463119,0.198556
4,7.0,Ac_Aj,32.883071,69.594801,NaN,0.700197,1.0,1.0,NaN,1.0,30.782857,1499.714286,3.827407,0.208470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,91.0,Ac_Aj_Ap,42.422443,59.248103,3.406041,2.095253,1.0,1.0,1.0,1.0,31.016364,1499.454545,18.419716,0.354792
60,92.0,Ac_Aj,25.416568,67.141218,NaN,0.587055,1.0,1.0,NaN,1.0,31.082143,2000.428571,17.138835,0.335360
61,94.0,Ac_Aj,34.890651,55.221670,NaN,1.344051,1.0,1.0,NaN,1.0,32.472727,2000.363636,4.957501,0.210718
62,96.0,Ac_Aj,76.408510,47.950683,NaN,2.351789,1.0,1.0,NaN,1.0,30.870769,1999.692308,41.477016,0.876901
